In [1]:
# 01-EDA-2.ipynb

# =====================================================================
# Cell 1: Imports and Global Settings
# =====================================================================
import pandas as pd

# Set pandas to show all columns in text outputs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)  # limit row prints to avoid huge spam

print("Notebook: 01-EDA-2 for synthetic VeriShield data with enhanced checks.")

Notebook: 01_EDA for synthetic VeriShield data with enhanced checks.


In [2]:
# =====================================================================
# Cell 2: Load CSVs
# =====================================================================
# Change the paths if needed; here we assume they're in the same directory
PATH_USERS = "/Users/harshil/Development/personal_projects/VeriShield-ML-Experiments/data_generators/synthetic_users.csv"
PATH_BUSINESSES = "/Users/harshil/Development/personal_projects/VeriShield-ML-Experiments/data_generators/synthetic_businesses.csv"
PATH_RELATIONSHIPS = "/Users/harshil/Development/personal_projects/VeriShield-ML-Experiments/data_generators/user_business_relationships.csv"

df_users = pd.read_csv(PATH_USERS)
df_businesses = pd.read_csv(PATH_BUSINESSES)
df_relationships = pd.read_csv(PATH_RELATIONSHIPS)

print("DataFrames loaded:")
print(f"  Users shape: {df_users.shape}")
print(f"  Businesses shape: {df_businesses.shape}")
print(f"  Relationships shape: {df_relationships.shape}")

DataFrames loaded:
  Users shape: (1500000, 13)
  Businesses shape: (150000, 6)
  Relationships shape: (3300622, 2)


In [3]:
# =====================================================================
# Cell 3: Inspect df_users
# =====================================================================
print("\n=== Users: .info() ===")
df_users.info()

print("\n=== Users: .head(5) ===")
print(df_users.head(5))

print("\n=== Users: .describe() (numeric/time fields) ===")
print(df_users.describe(include=[int, float, 'datetime']))

print("\n=== Users: .describe(include='object') ===")
print(df_users.describe(include=[object]))

print("\n=== Users: Checking for missing values by column ===")
print(df_users.isnull().sum())

print("\n=== Users: Fraud Label Counts ===")
fraud_label_counts = df_users['fraud_label'].value_counts(dropna=False)
print(fraud_label_counts)

# FRAUD RATIO
fraud_ratio_users = (df_users['fraud_label'] == 1).mean()
print(f"\nUsers Fraud Ratio: {fraud_ratio_users:.4%}")

print("\n=== Users: Sample Country Code Frequency (top 10) ===")
if 'country_code' in df_users.columns:
    print(df_users['country_code'].value_counts(dropna=False).head(10))
else:
    print("No 'country_code' column found.")

print("\n=== Users: Gender Frequency (if applicable) ===")
if 'gender' in df_users.columns:
    print(df_users['gender'].value_counts(dropna=False))
else:
    print("No 'gender' column found.")

print("\n=== Users: Top 10 IPs by frequency ===")
if 'signup_ip' in df_users.columns:
    ip_counts = df_users['signup_ip'].value_counts().head(10)
    print(ip_counts)
else:
    print("No 'signup_ip' column found.")

print("\n=== Users: Top 10 Email Domains ===")
if 'email' in df_users.columns:
    # If not yet added, create a quick domain extraction
    df_users['email_domain'] = df_users['email'].apply(lambda x: x.split('@')[-1] if x and '@' in str(x) else 'missing')
    print(df_users['email_domain'].value_counts().head(10))
else:
    print("No 'email' column found.")

# Additional columns introduced in enhanced data generator:
print("\n=== Users: Checking for 'device_id' ===")
if 'device_id' in df_users.columns:
    print("device_id found. Sample of device IDs:")
    print(df_users['device_id'].value_counts().head(10))
else:
    print("No 'device_id' column found.")

print("\n=== Users: Checking for 'burst_signup' (time-based suspicious flag) ===")
if 'burst_signup' in df_users.columns:
    print(df_users['burst_signup'].value_counts(dropna=False))
else:
    print("No 'burst_signup' column found.")


=== Users: .info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 13 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   user_id       1500000 non-null  int64 
 1   name          1470163 non-null  object
 2   email         1470088 non-null  object
 3   username      1500000 non-null  object
 4   birthdate     1500000 non-null  object
 5   gender        1500000 non-null  object
 6   signup_ip     1500000 non-null  object
 7   device_id     1500000 non-null  object
 8   phone         1469907 non-null  object
 9   country_code  1462545 non-null  object
 10  created_at    1500000 non-null  object
 11  burst_signup  1500000 non-null  bool  
 12  fraud_label   1500000 non-null  int64 
dtypes: bool(1), int64(2), object(10)
memory usage: 138.8+ MB

=== Users: .head(5) ===
   user_id             name                      email        username  \
0        1  Kelly Rodriguez       lisapark

In [4]:
# =====================================================================
# Cell 4: More advanced grouping on df_users
# =====================================================================
print("\n=== Group by 'fraud_label' and 'country_code', show top 10 combinations ===")
if 'country_code' in df_users.columns:
    user_grouped = df_users.groupby(['fraud_label', 'country_code']).size().reset_index(name='count')
    user_grouped_sorted = user_grouped.sort_values('count', ascending=False).head(10)
    print(user_grouped_sorted)
else:
    print("Skipping group by fraud_label & country_code (column not found).")

# IP-based grouping if we suspect collisions or repeated IP usage
if 'signup_ip' in df_users.columns:
    print("\n=== Group by 'fraud_label' and 'signup_ip', show top suspicious IPs (top 5) ===")
    ip_grouped = df_users.groupby(['fraud_label','signup_ip']).size().reset_index(name='count')
    ip_sorted = ip_grouped.sort_values('count', ascending=False).head(5)
    print(ip_sorted)

# For device_id collisions, if present:
if 'device_id' in df_users.columns:
    print("\n=== Group by 'device_id' to see if multiple users share same device ===")
    device_grouped = df_users.groupby('device_id').size().reset_index(name='count')
    print(device_grouped.sort_values('count', ascending=False).head(10))
    avg_device_ownership = device_grouped['count'].mean()
    print(f"Average users per device_id: {avg_device_ownership:.2f}")


=== Group by 'fraud_label' and 'country_code', show top 10 combinations ===
     fraud_label country_code  count
181            0           US   4845
154            0           SG   4820
31             0           CG   4813
149            0           SA   4808
75             0           ID   4804
170            0           TL   4788
172            0           TN   4785
12             0           BB   4780
46             0           DM   4774
47             0           DO   4770

=== Group by 'fraud_label' and 'signup_ip', show top suspicious IPs (top 5) ===
        fraud_label       signup_ip  count
285025            0  183.222.227.43    424
182988            0  158.232.214.93    417
316594            0  191.160.44.120    414
241309            0    172.41.15.47    414
553347            0  221.140.141.48    412

=== Group by 'device_id' to see if multiple users share same device ===
               device_id  count
0        device_AAAAnbsB      1
1000007  device_iiAuCfRc      1
1000005 

In [5]:
# =====================================================================
# Cell 5: Inspect df_businesses
# =====================================================================
print("\n=== Businesses: .info() ===")
df_businesses.info()

print("\n=== Businesses: .head(5) ===")
print(df_businesses.head(5))

print("\n=== Businesses: .describe() (numeric/time fields) ===")
print(df_businesses.describe(include=[int, float, 'datetime']))

print("\n=== Businesses: .describe(include='object') ===")
print(df_businesses.describe(include=[object]))

print("\n=== Businesses: Checking for missing values by column ===")
print(df_businesses.isnull().sum())

print("\n=== Businesses: Fraud Label Counts ===")
biz_fraud_counts = df_businesses['fraud_label'].value_counts(dropna=False)
print(biz_fraud_counts)

# FRAUD RATIO FOR BUSINESSES
fraud_ratio_biz = (df_businesses['fraud_label'] == 1).mean()
print(f"\nBusinesses Fraud Ratio: {fraud_ratio_biz:.4%}")

print("\n=== Businesses: Registration Country Frequency (top 10) ===")
if 'registration_country' in df_businesses.columns:
    print(df_businesses['registration_country'].value_counts(dropna=False).head(10))
else:
    print("No 'registration_country' column found.")

if 'owner_name' in df_businesses.columns:
    print("\n=== Businesses: Top 10 Owner Names by frequency ===")
    print(df_businesses['owner_name'].value_counts().head(10))


=== Businesses: .info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   business_id           150000 non-null  int64 
 1   business_name         150000 non-null  object
 2   registration_country  146173 non-null  object
 3   incorporation_date    150000 non-null  object
 4   owner_name            147013 non-null  object
 5   fraud_label           150000 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.9+ MB

=== Businesses: .head(5) ===
   business_id                    business_name registration_country  \
0            1                         Lee-Lamb                   TW   
1            2       Phantom Inc LightSlateGray                   NE   
2            3  matrix cross-media deliverables                   RW   
3            4                              Inc                   AF   
4            

In [6]:
# =====================================================================
# Cell 6: Inspect df_relationships
# =====================================================================
print("\n=== Relationships: .info() ===")
df_relationships.info()

print("\n=== Relationships: .head(5) ===")
print(df_relationships.head(5))

print("\n=== Relationships: .describe() ===")
print(df_relationships.describe())

unique_users_in_rels = df_relationships['user_id'].nunique()
unique_biz_in_rels = df_relationships['business_id'].nunique()
print(f"\nUnique user_ids in relationships: {unique_users_in_rels}")
print(f"Unique business_ids in relationships: {unique_biz_in_rels}")

# Check if some user_ids or business_ids appear outside valid range
print("\n=== Relationship Range Checks ===")
min_user_id = df_relationships['user_id'].min()
max_user_id = df_relationships['user_id'].max()
min_biz_id = df_relationships['business_id'].min()
max_biz_id = df_relationships['business_id'].max()
print(f"User ID range in relationships: {min_user_id} to {max_user_id}")
print(f"Business ID range in relationships: {min_biz_id} to {max_biz_id}")


=== Relationships: .info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3300622 entries, 0 to 3300621
Data columns (total 2 columns):
 #   Column       Dtype
---  ------       -----
 0   user_id      int64
 1   business_id  int64
dtypes: int64(2)
memory usage: 50.4 MB

=== Relationships: .head(5) ===
   user_id  business_id
0        2       124833
1        2       147503
2        2        89591
3        2       118427
4        2       141742

=== Relationships: .describe() ===
            user_id   business_id
count  3.300622e+06  3.300622e+06
mean   7.507134e+05  7.500224e+04
std    4.331510e+05  4.328681e+04
min    2.000000e+00  1.000000e+00
25%    3.760368e+05  3.751900e+04
50%    7.505840e+05  7.498300e+04
75%    1.126165e+06  1.124780e+05
max    1.499999e+06  1.500000e+05

Unique user_ids in relationships: 600072
Unique business_ids in relationships: 150000

=== Relationship Range Checks ===
User ID range in relationships: 2 to 1499999
Business ID range in relationships:

In [7]:
# =====================================================================
# Cell 7: Relationship distributions
# =====================================================================
# How many businesses does each user own, on average?
rel_count_by_user = df_relationships.groupby('user_id').size().reset_index(name='num_businesses')
print("\n=== Top 10 users by number of businesses owned ===")
print(rel_count_by_user.sort_values('num_businesses', ascending=False).head(10))

avg_biz_per_user = rel_count_by_user['num_businesses'].mean()
print(f"\nAverage businesses owned per user: {avg_biz_per_user:.2f}")

# How many users own each business?
rel_count_by_biz = df_relationships.groupby('business_id').size().reset_index(name='num_owners')
print("\n=== Top 10 businesses by number of owners ===")
print(rel_count_by_biz.sort_values('num_owners', ascending=False).head(10))

avg_users_per_biz = rel_count_by_biz['num_owners'].mean()
print(f"\nAverage owners per business: {avg_users_per_biz:.2f}")

# (Optional) Cross-check ownership by fraud
print("\n=== Optional: Merge relationships with user + business fraud to see correlation ===")
df_rels_users = df_relationships.merge(
    df_users[['user_id','fraud_label']], on='user_id', how='left'
).rename(columns={'fraud_label':'user_fraud'})

df_merged = df_rels_users.merge(
    df_businesses[['business_id','fraud_label']], on='business_id', how='left'
).rename(columns={'fraud_label':'biz_fraud'})

print(f"Merged shape: {df_merged.shape}. Head(5):")
print(df_merged.head(5))

# Quick cross-tab
print("\n=== Cross-tab of user_fraud vs. biz_fraud (first 10 rows) ===")
ctab = pd.crosstab(df_merged['user_fraud'], df_merged['biz_fraud'], margins=True).head(10)
print(ctab)

print("\nDone with extended EDA checks.")



=== Top 10 users by number of businesses owned ===
        user_id  num_businesses
359373   899168              10
499930  1250429              10
566854  1417291              10
95409    238632              10
444184  1111076              10
521089  1303313              10
73802    184542              10
419720  1049813              10
499928  1250425              10
444173  1111051              10

Average businesses owned per user: 5.50

=== Top 10 businesses by number of owners ===
        business_id  num_owners
22724         22725          50
80252         80253          45
134727       134728          44
75626         75627          43
103831       103832          42
143491       143492          42
25825         25826          42
97064         97065          42
49118         49119          42
21283         21284          42

Average owners per business: 22.00

=== Optional: Merge relationships with user + business fraud to see correlation ===
Merged shape: (3300622, 4). Head(5)